In [ ]:
import mysql.connector
import pandas as pd
import getpass  # To securely enter the password

def rentals_month(month: int, year: int, password: str, host="localhost", user="root", database="sakila", port=3306):
    """
    Retrieves rental data for a given month and year from the Sakila database.
    
    Parameters:
        month (int): The month (1-12) for which to retrieve rentals.
        year (int): The year (e.g., 2023) for which to retrieve rentals.
        password (str): The MySQL root password.
        host (str): MySQL server host (default: "localhost").
        user (str): MySQL username (default: "root").
        database (str): The database name (default: "sakila").
        port (int): The MySQL port (default: 3306).
    
    Returns:
        pd.DataFrame: A DataFrame containing rental records for the specified month and year.
    """

    try:
        # Connect to MySQL
        conn = mysql.connector.connect(
            host=host,
            user=user,
            password=password,
            database=database,
            port=port
        )

        # Create SQL query
        query = """
        SELECT rental_id, rental_date, inventory_id, customer_id, return_date, staff_id 
        FROM rental
        WHERE MONTH(rental_date) = %s AND YEAR(rental_date) = %s;
        """

        # Read data into Pandas DataFrame
        df = pd.read_sql(query, conn, params=(month, year))

        # Close connection
        conn.close()
        
        return df

    except mysql.connector.Error as err:
        print(f"Error: {err}")
        return None

# Example usage:
password = getpass.getpass("Enter MySQL password: ")  # Secure password input
df = rentals_month(5, 2005, password)

# Display DataFrame
print(df.head())


In [13]:
import mysql.connector
import pandas as pd

def rentals_month(conn, month: int, year: int):
    """
    Retrieves rental data for a given month and year from the Sakila database.

    Parameters:
        conn: MySQL Connector connection object.
        month (int): The month (1-12) for which to retrieve rentals.
        year (int): The year (e.g., 2005) for which to retrieve rentals.

    Returns:
        pd.DataFrame: A DataFrame containing rental records for the specified month and year.
    """

    # Define SQL query
    query = """
    SELECT rental_id, rental_date, inventory_id, customer_id, return_date, staff_id
    FROM rental
    WHERE MONTH(rental_date) = %s AND YEAR(rental_date) = %s;
    """

    # Execute query and load results into DataFrame
    cursor = conn.cursor()
    cursor.execute(query, (month, year))
    rows = cursor.fetchall()

    # Fetch column names
    columns = [col[0] for col in cursor.description]

    # Convert result to Pandas DataFrame
    df = pd.DataFrame(rows, columns=columns)

    # Close cursor
    cursor.close()

    return df


In [14]:
import mysql.connector
import getpass

# Secure password input
password = getpass.getpass("Enter MySQL password: ")

# Establish connection to MySQL
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password=password,
    database="sakila",
    port=3306
)

# Retrieve rental data for May 2005
df = rentals_month(conn, 5, 2005)

# Display results
print(df.head())

# Close the database connection
conn.close()


   rental_id         rental_date  inventory_id  customer_id  \
0          1 2005-05-24 22:53:30           367          130   
1          2 2005-05-24 22:54:33          1525          459   
2          3 2005-05-24 23:03:39          1711          408   
3          4 2005-05-24 23:04:41          2452          333   
4          5 2005-05-24 23:05:21          2079          222   

          return_date  staff_id  
0 2005-05-26 22:04:30         1  
1 2005-05-28 19:40:33         1  
2 2005-06-01 22:12:39         1  
3 2005-06-03 01:43:41         2  
4 2005-06-02 04:33:21         1  


In [15]:
import pandas as pd

def rental_count_month(df, month: int, year: int):
    """
    Calculates the number of rentals made by each customer in the specified month and year.

    Parameters:
        df (pd.DataFrame): The DataFrame containing rental data.
        month (int): The month (1-12) for which to calculate rental counts.
        year (int): The year (e.g., 2005) for which to calculate rental counts.

    Returns:
        pd.DataFrame: A DataFrame with customer_id and the rental count for the specified month and year.
    """
    
    # Group by customer_id and count rentals
    rental_count = df.groupby('customer_id').size().reset_index(name='rental_count')
    
    # Create dynamic column name based on month and year
    column_name = f"rentals_{month:02d}_{year}"
    
    # Rename the 'rental_count' column to the dynamic name
    rental_count = rental_count.rename(columns={'rental_count': column_name})
    
    return rental_count


In [16]:
import mysql.connector
import getpass
import pandas as pd

# Secure password input
password = getpass.getpass("Enter MySQL password: ")

# Establish connection to MySQL
engine = mysql.connector.connect(
    host="localhost",
    user="root",
    password=password,
    database="sakila",
    port=3306
)

# Retrieve rental data for May 2005
df = rentals_month(engine, 5, 2005)

# Calculate rental count by customer for May 2005
rental_count_df = rental_count_month(df, 5, 2005)

# Display the result
print(rental_count_df.head())

# Close the connection
engine.close()


   customer_id  rentals_05_2005
0            1                2
1            2                1
2            3                2
3            5                3
4            6                3


In [21]:
import pandas as pd

def compare_rentals(df1, df2, column_name_1, column_name_2):
    """
    Compares the number of rentals made by each customer in two DataFrames
    and returns a combined DataFrame with a 'difference' column.

    Parameters:
        df1 (pd.DataFrame): The first DataFrame containing customer rental counts for a specific month/year.
        df2 (pd.DataFrame): The second DataFrame containing customer rental counts for another month/year.
        column_name_1 (str): The name of the rental count column in the first DataFrame.
        column_name_2 (str): The name of the rental count column in the second DataFrame.

    Returns:
        pd.DataFrame: A DataFrame containing the 'customer_id', 'rentals_month_1', 
                       'rentals_month_2', and 'difference' (difference in rentals).
    """
    
    # Merge the two DataFrames on 'customer_id' (assuming they both have 'customer_id' column)
    combined_df = pd.merge(df1, df2, on='customer_id', how='outer', suffixes=('_month_1', '_month_2'))
    
    # Ensure correct renaming of the columns
    combined_df.rename(columns={column_name_1: 'rentals_month_1', column_name_2: 'rentals_month_2'}, inplace=True)
    
    # Calculate the difference between the rental counts
    combined_df['difference'] = combined_df['rentals_month_2'] - combined_df['rentals_month_1']
    
    return combined_df


In [22]:
# Example DataFrames for rentals in two different months
df_month_1 = pd.DataFrame({
    'customer_id': [1, 2, 3],
    'rentals_05_2005': [15, 12, 8]
})

df_month_2 = pd.DataFrame({
    'customer_id': [1, 2, 4],
    'rentals_06_2005': [18, 10, 20]
})

# Compare rentals between May and June 2005
comparison_df = compare_rentals(df_month_1, df_month_2, 'rentals_05_2005', 'rentals_06_2005')

# Display the result
print(comparison_df)


   customer_id  rentals_month_1  rentals_month_2  difference
0            1             15.0             18.0         3.0
1            2             12.0             10.0        -2.0
2            3              8.0              NaN         NaN
3            4              NaN             20.0         NaN
